# 7.01 Lesson 1 key concepts
## Reading the data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', None)


In [ ]:
data = pd.read_csv('learningSet.csv')


In [ ]:
print(data.shape)
data.head()

In [ ]:
data['TARGET_B'].value_counts()
# B = did they donate or not
# either 0 or 1

In [ ]:
data['TARGET_D'].describe()
# numerical target
# mean donation: 0.79$ -> so low, since most people did not donate
# D = how much did they donate

In [ ]:
95412 * (0.793 - 0.68) # (mean donation for all people minus cost of mailing piece) * number of mailing pieces
# 0.68 is fix 

In [ ]:
data[data['TARGET_B']>0]['TARGET_D'].describe()

In [ ]:
4843 * (15.62 - 0.68) # (mean donation for all people who made a donation minus cost of mailing piece) * number of mailing pieces 

To solve this problem we will first build a classification model to predict who will more likely respond and then for those respondents, we will build a regression model to predict the donation amount.

Then we can use the cost matrix to calculate the total benefit from the donations

Some of the challenges with the dataset are as follows:

Large number of features: The data set has over 450 features. Hence selecting the right features for the model is very critical and at the same time it is not easy as the same traditional ways of removing features is not effective given the large number of features. Apart from feature selection, feature extraction (creating your own features using the existing features) is also not easy in this case.
Sparsity of the dataset: There are a lot of features with a large number of null values.
Data imbalance: For developing a classification, there is a huge imbalance in the training dataset with only approximately 5000 values for one category as compared to cover 95,000 instances for the other category.

# 7.01 Lesson 2 key concepts
## Review data cleaning process

There are a lot of columns that have a very high percentage of null values. It is a highly sparse dataset. 
We can decide on a threshold and then remove those variables. There is no rule of thumb to decide on this threshold value. 
Sometimes it can as low as 25%-30%. And sometimes in some data sets you might find that even though there are more than 50% missing values in a column, you might have to include that variable in your analysis. 
A lot of it depends on the business context as well. In this case we will take this threshold to be 25% and then check the definitions of the columns filtered, to see if there is any column that we might want to keep.

In [ ]:
data.isna().sum()/len(data)

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
# nulls_percent_df
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].head(60)

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']
# choice: columns that we could potentially drop

In [ ]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

From the list above that includes the columns that have over 25% null values, you discussed with your manager you were told that the following columns are important -> wealth1, wealth2
We will remove these variables from the above list 
RDATE3, RAMNT_3 are important but they have too many null values 

# 7.01 Activity 2
## Remove columns with null values greater than a specified threshold

In [ ]:
drop_columns_list

In [ ]:
# del drop_columns_list[1]
# print("After deleting the item :", drop_columns_list)

In [ ]:
cols = ['WEALTH1','WEALTH2']
for item in cols:
    drop_columns_list.remove(item)
print(drop_columns_list)

# you can put multiple items in cols[]

In [ ]:
data = data.drop(columns=drop_columns_list)
data.shape

## More data cleaning

In [ ]:
data.head()

We can see that there are a lot of columns that have blank spaces which represent no value in this case. 
They were not identified as null values by python as they are empty spaces that are read as character values by 
python. We will replace those values by NaNs and repeat the analysis 

Before we do that we will replace the blank values from the column "MAILCODE" by "A" which would mean the address is okay (pl check the definition of the variable in the description)

In [ ]:
data['MAILCODE'].value_counts()#.index

In [ ]:
data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x.replace(" ", "A"))

In [ ]:
# Now we can replace the rest space characters with np.NaN

In [ ]:
data = data.apply(lambda x: x.replace(" ", np.NaN))

In [ ]:
data.head()

# 7.01 Activity 3
## Remove columns with null values greater than a specified threshold

In [ ]:
data.isna().sum()/len(data)

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
# nulls_percent_df
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0].head(60)

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25]
columns_above_threshold['column_name']

In [ ]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

We would again repeat the same exercise as the last time. We will discuss it with the team, manager, and/or other stakeholders to see which columns we need to retain here. 

Like last time we will keep the following -> wealth1, wealth2 along with these VETERANS, SOLIH

In [ ]:
cols = ['WEALTH1','WEALTH2','VETERANS','SOLIH']
for item in cols:
    drop_columns_list.remove(item)
print(drop_columns_list)


In [ ]:
data = data.drop(columns=drop_columns_list)
data.shape

In [ ]:
data.head()

# 7.01 Lesson 4 key concepts
## Separating Target, Numerical Features, and Categorical Features

Since we have a huge number of features, it would be easier to work independently on numerical features and categorical features.

For the target variables, for now we will retain them both together. But later, we will build a classification model first where we would need the column TARGET_B only.

In [ ]:
Y = data[['TARGET_B', 'TARGET_D']]
Y.head()

In [ ]:
numerical = data.select_dtypes(np.number)
numerical = numerical.drop(columns = ['TARGET_B', 'TARGET_D'])
numerical.head()

In [ ]:
numerical.shape

In [ ]:
categorical = data.select_dtypes(np.object)
categorical.head()

In [ ]:
categorical.shape

## Working with categorical columns

We will work with the categorical features first. Look at the columns one by one. Some of the operations which we will perform are:

- Replace null values with the most occurring categories
- Reduce the number of categories in a column by grouping

It is important to note that some columns are defined by python as categorical/object types. There might be other columns defined as numerical that we want as categorical. We will look them later when we were working on numerical types.

In [ ]:
categorical.isna().sum()

## Categorical Variables

Here we will try to reduce the number of categories. An ideal way would have been to group the states into 
regions. But in this case we will group all the states with counts less than 2500 into one category "other"

In [ ]:
categorical.isna().sum()/len(categorical)

In [ ]:
pd.DataFrame(categorical['STATE'].value_counts()).reset_index()

In [ ]:
df = pd.DataFrame(categorical['STATE'].value_counts()).reset_index()

# states that have less than 2500 represantatives in our data
df.columns = ['state', 'count']
other_states = list(df[df['count']<2500]['state'])
other_states

def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x
    
categorical['STATE'] = list(map(clean_state, categorical['STATE']))

In [ ]:
categorical['STATE'].value_counts()

# 7.01 Lab | Revisiting Machine Learning Case Study

ADD HERE CODE FOR LAB

In [ ]:
drop_list = []

In [ ]:
categorical['OSOURCE']

In [ ]:
drop_list.append('OSOURCE')

In [ ]:
categorical['ZIP']

In [ ]:
drop_list.append('ZIP')

# 7.02

We will work on the column DOMAIN. Let's replace the null values with the category that is represented the most and then we will split the data into two columns DOMAIN_A and DOMAIN_B.

- DOMAIN_A will consist of the first character from the DOMAIN
- DOMAIN_B will consist of the second character from the DOMAIN

Then we will drop the original column DOMAIN

In [ ]:
categorical['DOMAIN'].value_counts(dropna=False)

In [ ]:
categorical['DOMAIN'] = categorical['DOMAIN'].fillna('R2')
# fill in most frequent value

In [ ]:
categorical['DOMAIN_A'] = list(map(lambda x: x[0], categorical['DOMAIN']))
# lambda function applied on whole domain. function on first element
categorical['DOMAIN_B'] = list(map(lambda x: x[1], categorical['DOMAIN']))
# function on second element

In [ ]:
categorical = categorical.drop(columns=['DOMAIN'])

In [ ]:
categorical.head()

### Activity 2.1

Use the method value_counts on the columns MAILCODE, NOEXCH, and MDMAUD and check the proportion of category representation in those columns. Since there is a huge imbalance in the representation of category/categories, we will remove this column. Add those columns to the drop_list.

In [ ]:
drop_list=[]

In [ ]:
categorical['MAILCODE'].value_counts()

In [ ]:
drop_list.append('MAILCODE')
#rememeber to check mail code before actually sending out mail pieces

In [ ]:
categorical['NOEXCH'].value_counts(dropna=False)#.index

In [ ]:
drop_list.append('NOEXCH')
# noexch do not exchange address

In [ ]:
print(categorical['MDMAUD'].value_counts())
# since most of the data is unknown here we will drop this column

In [ ]:
drop_list.append('MDMAUD')

In [ ]:
# MDMAUD_XXX are also like MDMAUD 
# recency
# frequency
# amount

In [ ]:
categorical['MDMAUD_R'].value_counts()

In [ ]:
drop_list= drop_list + ['MDMAUD_R', 'MDMAUD_F','MDMAUD_A']

REPLACING NULL VALUES

In [ ]:
categorical['CLUSTER'].value_counts(dropna=False)

In [ ]:
categorical['CLUSTER'].sort_values().unique()

In [ ]:
categorical['CLUSTER'].unique()

In [ ]:
# sorted(categorical['CLUSTER'].unique())

In [ ]:
sorted(categorical['CLUSTER'].value_counts().index)

In [ ]:
categorical['CLUSTER'] = categorical['CLUSTER'].fillna('40') 
# 40 major 'class'
# 'other' would also be a valid choice

In [ ]:
categorical['HOMEOWNR'].value_counts(dropna=False)

In [ ]:
categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna('U')
# assumption: NaN also means 'we don't know'

In [ ]:
categorical['GENDER'].value_counts(dropna=False)

In [ ]:
# discuss whether this makes more sense than the alternatives
categorical['GENDER'] = categorical['GENDER'].fillna('F')

In [ ]:
#End Activity

MDMAUD_XXX are also like MDMAUD



In [ ]:
# categorical['MDMAUD_R'].value_counts()

In [ ]:
# drop_list = drop_list + ['MDMAUD_R', 'MDMAUD_F','MDMAUD_A']

Replacing NULL values

In [ ]:
# categorical['CLUSTER'].value_counts(dropna=False)

In [ ]:
# categorical['CLUSTER'] = categorical['CLUSTER'].fillna('40')

In [ ]:
# categorical['HOMEOWNR'].value_counts(dropna=False)

In [ ]:
# categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna('U')

In [ ]:
# categorical['GENDER'].value_counts(dropna=False)

In [ ]:
#discuss why this makes more sense than the alternatives
# categorical['GENDER'] = categorical['GENDER'].fillna('F')

### Activity 2.2.1

Replace null values in the columns DATASRCE and GEOCODE2

In [ ]:
categorical['DATASRCE'].value_counts(dropna=False)

In [ ]:
categorical['DATASRCE'] = categorical['DATASRCE'].fillna('3')

In [ ]:
categorical['GEOCODE2'].value_counts(dropna=False)

In [ ]:
categorical['GEOCODE2'] = categorical['GEOCODE2'].fillna('A')

In [ ]:
categorical.head()

In [ ]:
print(categorical['DATASRCE'])

In [ ]:
print(categorical['GEOCODE2'])
# fill 

Removing columns with similar information

In [ ]:
categorical['RFA_2'].value_counts()

In [ ]:
# We will keep the column RFA_2X. We will delete rest of the columns 

for col_name in categorical.columns:
    if "RFA" in col_name:
        drop_list.append(col_name)       

In [ ]:
drop_list.remove('RFA_2R')
drop_list.remove('RFA_2A')
drop_list

In [ ]:
categorical = categorical.drop(columns=drop_list)
categorical.head()

In [ ]:
categorical.isna().sum()

In [ ]:
data['OSOURCE'].value_counts(dropna=False)

In [ ]:
data['OSOURCE'] = data['OSOURCE'].fillna('MBC')

In [ ]:
categorical = categorical.drop(columns=['OSOURCE', 'ZIP'])
categorical.head()

In [ ]:
# drop OSOURCE

# Working with numerical columns 

In [ ]:
numerical.head()

In [ ]:
numerical.shape

In [ ]:
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

### Activity 2.2.2

Remove the columns starting with ADATE_. We are assuming that the date when the previous mail was done is not significant in the respondents decision to give donation. They may or may not even remember when they received the mail in the previous years. And for the column ADATE_2, check the values in the column. If the values are pretty much the same, then remove this column as well.

In [ ]:
drop_list = []
for col_name in numerical.columns:
    if "ADATE" in col_name:
        drop_list.append(col_name)   

In [ ]:
numerical['ADATE_2'].value_counts()
# ADATE2 gives us the nearly the same value throughout -> not much information, so we drop it

### Filling NULL values with plots

In [ ]:
sns.distplot(numerical['AGE'])
plt.show()

In [ ]:
#numerical['AGE'] = numerical["AGE"].fillna(np.mean(numerical['AGE']))
sns.distplot(numerical['AGE'])
plt.show()
#probably would be a bit better to use a predictive method here but this works

In [ ]:
#sns.distplot(numerical['INCOME'])  # this will not work as there are NaNs in the column
sns.distplot(numerical[numerical['INCOME'].isna()==False]['INCOME']) 
plt.show()

In [ ]:
# looks like the variable is actually categorical. We can verify it by using value_counts()
print(numerical['INCOME'].value_counts())
numerical['INCOME'] = numerical['INCOME'].astype('object')

In [ ]:
numerical['INCOME'] = numerical['INCOME'].fillna('5.0') # Replacing the null values with the most represented categoty

In [ ]:
sns.distplot(numerical[numerical['CLUSTER2'].isna()==False]['CLUSTER2']) 
plt.show()

In [ ]:
numerical['CLUSTER2'] = numerical['CLUSTER2'].fillna(np.ceil(np.mean(numerical['CLUSTER2'])))

## Activity 2.3

In [ ]:
sns.distplot(numerical[numerical['WEALTH2'].isna()==False]['WEALTH2']) 
plt.show()

In [ ]:
numerical['WEALTH2'] = numerical['WEALTH2'].astype('object')
numerical['WEALTH2'] = numerical['WEALTH2'].fillna(9.0)

In [ ]:
sns.distplot(numerical[numerical['TIMELAG'].isna()==False]['TIMELAG']) 
plt.show()

In [ ]:
numerical['TIMELAG'] = numerical['TIMELAG'].fillna(np.ceil(np.mean(numerical['TIMELAG'])))

In [ ]:
sns.distplot(numerical[numerical['TIMELAG'].isna()==False]['TIMELAG']) 
plt.show()

In [ ]:
numerical.isnull().value_counts()

# Lab 2

ADD HERE CODE FOR LAB 2

In [ ]:
numerical.isnull().sum()

In [ ]:
nulls_percent_numerical = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
nulls_percent_numerical.columns = ['column_name', 'nulls_percentage']
nulls_percent_numerical

In [ ]:
nulls_percent_numerical[nulls_percent_numerical['nulls_percentage']!=0].head(60)

In [ ]:
numerical['WEALTH1'].value_counts()

In [ ]:
numerical['WEALTH1'].isna().sum()

In [ ]:
numerical['WEALTH1'] = numerical['WEALTH1'].astype("object")
numerical['WEALTH1'] = numerical['WEALTH1'].fillna(5.0)

In [ ]:
sns.distplot(numerical[numerical['WEALTH1'].isna()==False]['WEALTH1']) 
plt.show()

In [ ]:
columns_above_threshold = nulls_percent_numerical[nulls_percent_numerical['nulls_percentage']>0.85]
columns_above_threshold['column_name']

In [ ]:
drop_list.append('SOLIH')
drop_list.append('VETERANS')

In [ ]:
print(categorical['GENDER'].value_counts())
categorical['GENDER'] = categorical['GENDER'].fillna('F')

In [ ]:
categorical['GENDER'].value_counts()

In [ ]:
categorical['GENDER'].isna().sum()

In [ ]:
def clean(a):
    if a == 'F':
        return 'F'
    elif a == 'M':
        return 'M'
    else:
        return 'Other'

categorical['GENDER'] = categorical['GENDER'].apply(clean)

In [ ]:
categorical['GENDER'].value_counts()

In [ ]:
numerical['WEALTH1']

In [ ]:
# clean column ADI
print(numerical['ADI'].value_counts())
numerical['ADI'] = numerical['ADI'].astype('object')

In [ ]:
numerical['ADI'].isna().sum()

In [ ]:
sns.distplot(numerical[numerical['ADI'].isna()==False]['ADI']) 
plt.show()
# all graphs (for ADI, DMA, MSA) look exactly the same
# the columns also have the same amount of NaN values
# assumption: they all give the same information
# method: drop NaNs for one column -> you dropped them for all the others as well

In [ ]:
numerical['DMA'].isna().sum()
numerical['MSA'].isna().sum()

In [ ]:
numerical = numerical[numerical['ADI'].isna()==False]

In [ ]:
numerical['DMA'].isna().sum()
# numerical['MSA'].isna().sum()

# Wrapping up

In [ ]:
numerical = numerical.drop(columns=drop_list)

In [ ]:
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

In [ ]:
numerical.to_csv('numerical7_02.csv')
categorical.to_csv('categorical7_02.csv')

In [ ]:
Y.to_csv('target7_02.csv')

In [ ]:
# alternative
# numerical.to_pickle('numerical7_02.p')
# categorical.to_pickle('categorical7_02.p')
# Y.to_pickle('target7.02.p')